In [1]:
import numpy as np

import cv2

import os
import glob
import re

import argparse

In [2]:
def main(img_dir, save_dir, phase):
    # make image list
    if phase=='train':
        img_list = glob.glob(f'../input/raw/{phase}/{img_dir}/*.jpeg')
    
    if phase=='test':
        img_list = glob.glob(f'../input/raw/{phase}/*.jpeg')
    #print(img_list)
    
    # save model path
    if phase=='train':
        SAVE_MODEL_PATH = f'../input/{save_dir}/{phase}/{img_dir}/'
    if phase=='test':
        SAVE_MODEL_PATH = f'../input/{save_dir}/{phase}/'
    os.makedirs(SAVE_MODEL_PATH, exist_ok=True)
    print(SAVE_MODEL_PATH)

    for image in img_list:

        # image file name
        if phase=='train':
            file_name = re.split('[/////]', image)[5]
        if phase=='test':
            file_name = re.split('[////]', image)[4]
        #print(file_name)

        # read an image
        img = cv2.imread(image)

        # make img_copy
        img_copy = img.copy()

        # gray scaling
        img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)

        # binarization using Otsu's method
        _, th = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # configure the kernel
        kernel = np.ones((100,100),np.uint8)

        # morphological transformation(Dilation)
        th_dilation = cv2.dilate(cv2.bitwise_not(th),kernel,iterations = 1)

        # contour extraction
        contours, _ = cv2.findContours(
            th_dilation,
            cv2.RETR_LIST,
            cv2.CHAIN_APPROX_NONE
        )

        # extract contours
        for i in range(len(contours)):
            if len(contours[i]) > 0:
                
                # remove small contours
                if cv2.contourArea(contours[i]) < 30000:
                    continue
                
                rect = contours[i]
                x, y, w, h = cv2.boundingRect(rect)
                #print(f'x: {x}, y:{y}, w:{w}, h:{h}')

                # which is bigger ? w? h? + 20 pixcel margin
                b = max(w, h)

                padding = 5
                
                # top, bottom, left, right
                top, bottom, left, right = y-padding, y+b+padding, x-padding, x+b+padding

        crop_img = img[top : bottom, left : right]

        # create a CLAHE object (Arguments are optional).
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

        b1,g1,r1 = cv2.split(crop_img)
        b2 = clahe.apply(b1)
        g2 = clahe.apply(g1)
        r2 = clahe.apply(r1)

        crop__flatten_img = cv2.merge((b2,g2,r2))
        
        cv2.imwrite(
            os.path.join(SAVE_MODEL_PATH, file_name),
            crop__flatten_img,
            [cv2.IMWRITE_JPEG_QUALITY, 100]
        )

In [7]:
# create cropped image
main(
    img_dir='regular',
    save_dir='crop_flatten',
    phase='test'
)

../input/crop_flatten/test/
